In [30]:
## 알고리즘 문제, 장애물이 큐에 들어가는지 등
## 고정 장애물 만나도 rescan

## detectability를 정규화할 수 있는 방법?
## topography & cost 1~10으로?

## 장애물이 갑자기 사라지는 현상?
## 범위 안에 들어왔을 때 무조건 잡는게 너무 빡센듯

In [1]:
import heapq
import sys
import numpy as np
import random
import pygame
import math
from itertools import combinations_with_replacement

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#'xnym' -> [n,m]
def stateNameToCoords(name):
    return [int(name.split('x')[1].split('y')[0]), int(name.split('x')[1].split('y')[1])]

'''왜 sort가 주석 처리 되어있는지?'''
def topKey(queue):
    queue.sort()
    # print(queue)
    if len(queue) > 0:
        return queue[0][:2] #queue가장 위에 있는 것의 k1,k2만 가져옴
    else:
        print('empty queue!')
        return (float('inf'), float('inf'))

def heuristic_from_s(Graph, id, s):
    id_coords=stateNameToCoords(id)
    s_coords=stateNameToCoords(s)
    
    x_distance =  (int(id_coords[0]) - int(s_coords[0]))**2 ##abs(int(id_coords[0]) - int(s_coords[0]))
    y_distance = (int(id_coords[1]) - int(s_coords[1]))**2 ##abs(int(id_coords[1]) - int(s_coords[1]))
    return np.sqrt(x_distance + y_distance)  ##x_distance + y_distance

def calculateKey(Graph, id, s_current, k_m):
    id_coords=stateNameToCoords(id)
    return (min(Graph.graph[id].g, Graph.graph[id].rhs) + heuristic_from_s(Graph, id, s_current) + k_m 
            + alpha * Graph.cells[id_coords[1]][id_coords[0]]
            + beta * Graph.topography[id_coords[1]][id_coords[0]], 
            min(Graph.graph[id].g, Graph.graph[id].rhs)
            + alpha * Graph.cells[id_coords[1]][id_coords[0]]
            + beta * Graph.topography[id_coords[1]][id_coords[0]]) 


def updateVertex(Graph, queue, id, s_current, k_m):
    #   s_goal = Graph.goal
    if id != Graph.goal:
        min_rhs = float('inf')
        for i in Graph.graph[id].children:
            min_rhs = min(min_rhs, Graph.graph[i].g + Graph.graph[id].children[i])
        Graph.graph[id].rhs = min_rhs
        
    id_in_queue = [item for item in queue if id in item]
    if id_in_queue != []:
        if len(id_in_queue) != 1:
            raise ValueError('more than one ' + id + ' in the queue!')
        queue.remove(id_in_queue[0])
    if Graph.graph[id].rhs != Graph.graph[id].g:
        '''장애물 위치도 queue에 들어가는지는 queue print해서 봐보자'''
        heapq.heappush(queue, calculateKey(Graph, id, s_current, k_m) + (id,))


def computeShortestPath(Graph, queue, s_start, k_m):
    while (Graph.graph[s_start].rhs != Graph.graph[s_start].g) or (topKey(queue) < calculateKey(Graph, s_start, s_start, k_m)):
        k_old = topKey(queue)
        u = heapq.heappop(queue)[2]
        
        if k_old < calculateKey(Graph, u, s_start, k_m):
            heapq.heappush(queue, calculateKey(Graph, u, s_start, k_m) + (u,))
        elif Graph.graph[u].g > Graph.graph[u].rhs:
            Graph.graph[u].g = Graph.graph[u].rhs
            for i in Graph.graph[u].parents:
                updateVertex(Graph, queue, i, s_start, k_m)
        else:
            Graph.graph[u].g = float('inf')
            updateVertex(Graph, queue, u, s_start, k_m)
            for i in Graph.graph[u].parents:
                updateVertex(Graph, queue, i, s_start, k_m)

def nextInShortestPath(Graph, s_current):
    min_rhs = float('inf')
    s_next = None
    if Graph.graph[s_current].rhs == float('inf'):
    #if Graph.graph[s_current].g == float('inf'):
        print('You are done stuck')
        return s_current
    else:
        for i in Graph.graph[s_current].children:
            # print(i)
            child_cost = Graph.graph[i].g + Graph.graph[s_current].children[i]
            # print(child_cost)
            if (child_cost) < min_rhs:
                min_rhs = child_cost
                s_next = i
        if s_next:
            return s_next
        else:
            #raise ValueError('could not find child for transition!')
            print('could not find child for transition!')
            return s_current

def scanForObstacles(Graph, queue, s_current, k_m):
    states_to_update = {}
    
    for neighbor in Graph.graph[s_current].children:
        neighbor_coords = stateNameToCoords(neighbor)
        states_to_update[neighbor] = Graph.cells[neighbor_coords[1]][neighbor_coords[0]]
        
    new_obstacle = False
    
    for state in states_to_update:
        if states_to_update[state] < 0:  # found cell with obstacle
            # print('found obstacle in ', state)
            for neighbor in Graph.graph[state].children:
                # first time to observe this obstacle where one wasn't before
                if(Graph.graph[state].children[neighbor] != float('inf')):
                    neighbor_coords = stateNameToCoords(state)
                    Graph.cells[neighbor_coords[1]][neighbor_coords[0]] = -2
                    Graph.graph[neighbor].children[state] = float('inf')
                    Graph.graph[state].children[neighbor] = float('inf')
                    updateVertex(Graph, queue, state, s_current, k_m)
                    new_obstacle = True
    return new_obstacle



def Move(Graph, s_current):
    if(s_current == Graph.goal):
        return 'goal'
    else:
        s_new = nextInShortestPath(Graph, s_current)
        new_coords = stateNameToCoords(s_new)
        if(Graph.cells[new_coords[1]][new_coords[0]] == -1):  # just ran into new obstacle
            s_new = s_current  # need to hold tight and scan/replan first
        
        return s_new
        
def Rescan(Graph, queue, s_current,k_m):
    results = scanForObstacles(Graph, queue, s_current, k_m)
    k_m += heuristic_from_s(Graph, s_current, Graph.goal)
    #s_last = s_current
    computeShortestPath(Graph, queue, s_current, k_m)

    return k_m
'''이전 moveAndRescan하고 의미가 달라지나?'''


def initDStarLite(Graph, queue, s_start, s_goal, k_m):
    Graph.graph[s_goal].rhs = 0
    heapq.heappush(queue, calculateKey(Graph, s_goal, s_start, k_m) + (s_goal,))
    #print(queue[0])
    computeShortestPath(Graph, queue, s_start, k_m)
    return (Graph, queue, k_m)

In [3]:
def L1(a,b):
    ret=np.abs(a[0]-b[0])+np.abs(a[1]-b[1])
    if ret==0:
        return 0.1
    else:
        return ret

def L2(a,b):
    ret=np.sqrt(np.power((a[0]-b[0]),2)+np.power((a[1]-b[1]),2))
    if ret==0:
        return 0.1
    else:
        return ret

def softmax(X):
    exp_a = np.exp(X)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

def get_move(move_prob):
    sample=np.random.rand()
    standard=move_prob[0]
    
    for i in range(len(move_prob)-1):
        if sample<standard:
            return i
        else:
            standard+=move_prob[i+1]
            
    return len(move_prob)-1

def move_obstacle(graph,pos_coords,move_chance=0.5,smart=False,detectability=1):
    if graph.obstacles:
        temp_obstacles=graph.obstacles.copy()
        for obstacle in graph.obstacles:
            if obstacle[2]:
                if random.random()<move_chance:
                    if smart:
                        move_prob=softmax([detectability*1/L2((pos_coords[1],pos_coords[0]),(obstacle[0][0]+i[0],obstacle[0][1]+i[1])) for i in move])
                        ## L1(대각선 이동 가능시 L2로 바꾸기) ## softmax로 확률 만들기 또는 그냥 합으로 나누기
                        dy,dx=move[get_move(move_prob)]
                            
                    else:
                        dy,dx=random.choice(move)
                        
                    new_x,new_y=obstacle[0][1]+dx,obstacle[0][0]+dy
                    if 0<=new_x<graph.x_dim and 0<=new_y<graph.y_dim and (new_y,new_x) not in [obs[0] for obs in temp_obstacles]:
                        temp_obstacles.remove(obstacle)
                        temp_obstacles.append(((new_y,new_x),obstacle[1],obstacle[2]))
                        
        graph.obstacles=temp_obstacles
        graph.setObstacle()

In [4]:
class Node:
    def __init__(self, id):
        self.id = id
        # dictionary of parent node ID's
        # key = id of parent
        # value = (edge cost,)
        self.parents = {}
        # dictionary of children node ID's
        # key = id of child
        # value = (edge cost,)
        self.children = {}
        # g approximation
        self.g = float('inf')
        # rhs value
        self.rhs = float('inf')

    def __str__(self):
        return 'Node: ' + self.id + ' g: ' + str(self.g) + ' rhs: ' + str(self.rhs)

    def __repr__(self):
        return self.__str__()

    def update_parents(self, parents):
        self.parents = parents


class Graph:
    def __init__(self):
        self.graph = {}

    def __str__(self):
        msg = 'Graph:'
        for i in self.graph:
            msg += '\n  node: ' + i + ' g: ' + str(self.graph[i].g) + ' rhs: ' + str(self.graph[i].rhs)
        return msg

    def __repr__(self):
        return self.__str__()

    def setStart(self, id):
        if(self.graph[id]):
            self.start = id
        else:
            raise ValueError('start id not in graph')

    def setGoal(self, id):
        if(self.graph[id]):
            self.goal = id
        else:
            raise ValueError('goal id not in graph')

In [5]:
#그래프 객체(장애물위치, 장애물cost, 지형지물 정보, 노드 정보 등)
class GridWorld(Graph):
    def __init__(self, x_dim, y_dim):
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.obstacles=[] #((y,x),cost)
        self.cells=np.zeros((y_dim,x_dim))
        #self.topography=np.random.randint(1,11, size=(y_dim, x_dim))
        #self.topography=np.random.rand(y_dim,x_dim)
        self.topography=[[random.choice(range(1, 10)) / 10 for _ in range(x_dim)] for _ in range(y_dim)]
        self.graph = {}
        self.generateGraphFromGrid()
        # self.printGrid()

    def __str__(self):
        msg = 'Graph:'
        for i in self.graph:
            msg += '\n  node: ' + i + ' g: ' + str(self.graph[i].g) + ' rhs: ' + str(self.graph[i].rhs) + ' neighbors: ' + str(self.graph[i].children)
        return msg

    def __repr__(self):
        return self.__str__()

    def printGrid(self):
        print('** GridWorld **')
        for row in self.cells:
            print(row)

    def printGValues(self):
        for j in range(self.y_dim):
            str_msg = ""
            for i in range(self.x_dim):
                node_id = 'x' + str(i) + 'y' + str(j)
                node = self.graph[node_id]
                if node.g == float('inf'):
                    str_msg += ' - '
                else:
                    str_msg += ' ' + str(node.g) + ' '
            print(str_msg)

    def generateGraphFromGrid(self):
        edge = 1
        cross=np.sqrt(2)
        for i in range(len(self.cells)):
            row = self.cells[i]
            for j in range(len(row)):
                # print('graph node ' + str(i) + ',' + str(j))
                node = Node('x' + str(j) + 'y' + str(i))
                if j > 0:  # not top row
                    node.parents['x' + str(j - 1) + 'y' + str(i)] = edge
                    node.children['x' + str(j - 1) + 'y' + str(i)] = edge
                if j + 1 < self.x_dim:  # not bottom row
                    node.parents['x' + str(j + 1) + 'y' + str(i)] = edge
                    node.children['x' + str(j + 1) + 'y' + str(i)] = edge
                if i > 0:  # not left col
                    node.parents['x' + str(j) + 'y' + str(i - 1)] = edge
                    node.children['x' + str(j) + 'y' + str(i - 1)] = edge
                if i + 1 < self.y_dim:  # not right col
                    node.parents['x' + str(j) + 'y' + str(i + 1)] = edge
                    node.children['x' + str(j) + 'y' + str(i + 1)] = edge
                if i > 0 and j > 0:
                    node.parents['x' + str(j - 1) + 'y' + str(i - 1)] = cross
                    node.children['x' + str(j - 1) + 'y' + str(i - 1)] = cross    
                if j > 0 and i + 1 < self.y_dim:
                    node.parents['x' + str(j - 1) + 'y' + str(i + 1)] = cross
                    node.children['x' + str(j - 1) + 'y' + str(i + 1)] = cross
                if j + 1 < self.x_dim and i > 0:
                    node.parents['x' + str(j + 1) + 'y' + str(i - 1)] = cross
                    node.children['x' + str(j + 1) + 'y' + str(i - 1)] = cross
                if j + 1 < self.x_dim and i + 1 < self.y_dim:
                    node.parents['x' + str(j + 1) + 'y' + str(i + 1)] = cross
                    node.children['x' + str(j + 1) + 'y' + str(i + 1)] = cross
                self.graph['x' + str(j) + 'y' + str(i)] = node
    
    #장애물 초기화 및 이동 반영
    def setObstacle(self):
        self.cells=np.zeros((self.y_dim,self.x_dim))
        for obs in self.obstacles:
            already=[]
            loc=obs[0]
            cost=obs[1]
                
            for i,c in enumerate(cost,start=1):
                if i==1:
                    candidate=move
                else:
                    candidate=[]
                    for a,*b in combinations_with_replacement(move, i):
                        candidate.append(tuple(sum(elem) for elem in zip(a, *b)))
                        
                    candidate=list(set(candidate)-set(already)-set([(0, 0)]))
                    
                already.extend(candidate)
                
                for m in candidate:
                    boundary=(loc[0]+m[0],loc[1]+m[1]) ##i배 할게 아니라 주변 다 삥 둘러서 cost줘야해

                    if [boundary[1],boundary[0]]==stateNameToCoords(self.goal):
                        continue

                    if (0<=boundary[0]<self.y_dim) and (0<=boundary[1]<self.x_dim): 
                        self.cells[boundary[0]][boundary[1]]+=c

        for obs in self.obstacles:
            loc=obs[0]
            self.cells[loc[0]][loc[1]]=-1

In [6]:
def initObstacles(Graph,obstacle_ratio,fixed_obs_ratio,influence=1):
    #obstacle grid크기에서 일정 비율만큼 랜덤 위치 생성 / obstacle cost 각각 랜덤 생성(0~1)
    max_obstacle_num=int(obstacle_ratio*Graph.x_dim*Graph.y_dim)
    fixed_obstacle_num=int(max_obstacle_num*fixed_obs_ratio)
    ob_num=0
    fix_num=0
    while ob_num<max_obstacle_num:
        ob_x=random.randint(0,Graph.x_dim-1)
        ob_y=random.randint(0,Graph.y_dim-1)
        
        if (ob_y,ob_x) in [obs[0] for obs in Graph.obstacles]:
            continue
        elif [ob_x,ob_y]==stateNameToCoords(Graph.goal):
            continue
        elif [ob_x,ob_y]==stateNameToCoords(Graph.start):
            continue
            
        if fix_num<fixed_obstacle_num:
            movable=0
            cost=0
            fix_num+=1
        else:
            movable=1
            #cost=random.randint(1,10)
            #cost=random.random()
            cost=random.choice(range(1, 10)) / 10
        
        cost_set=[]
        for i in range(1,influence+1):
            cost_set.append(cost/i)
        
        #print(((ob_y,ob_x),tuple(cost_set),movable))
        Graph.obstacles.append(((ob_y,ob_x),tuple(cost_set),movable))
        ob_num+=1
    
    Graph.setObstacle()

In [36]:
# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED   = (255, 0, 0)
GRAY = (145, 145, 102)
DARKGRAY= (100, 100, 100)
BLUE = (0, 0, 200)
PINK = (255,0,255)

#장애물 및 현재위치 이동 가능한 경로
move=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
#move=[(-1,0),(0,-1),(0,1),(1,0)]

def colors(grid):
    if grid==0:
        return WHITE
    if grid==-1:
        return RED
    if grid==-2:
        return BLACK
    #return max((255-int((255-255)*grid/2),255-int((255-204)*grid/2),255-int((255-102)*grid/2)),(255,204,102))
    return max((255-int((255-255)*grid),255-int((255-204)*grid),255-int((255-102)*grid)),(255,204,102))

# This sets the width and height of each grid location
width = 30
height = 30

# This sets the margin between each cell
margin = 2

dimension_x =10
dimension_y =10
VIEWING_RANGE = 1

# obstacle num ratio of grid
obstacle_ratio=0.1
fixed_obs_ratio=0.8
move_chance=0.9
alpha = 10 #obstacle cost
beta = 5 #topography
smart=True
step=2
influence=2
detectability=1

# Set the height and width of the screen
WINDOW_SIZE = [(width + margin) * dimension_x + margin,
               (height + margin) * dimension_y + margin]

def draw_arrow(screen, colour, start, end):
    pygame.draw.line(screen,colour,start,end,1)
    rotation = math.degrees(math.atan2(start[1]-end[1], end[0]-start[0]))+90
    rad=12
    pygame.draw.polygon(screen, BLACK, ((end[0]+rad*math.sin(math.radians(rotation)), end[1]+rad*math.cos(math.radians(rotation))), (end[0]+rad*math.sin(math.radians(rotation-120)), end[1]+rad*math.cos(math.radians(rotation-120))), (end[0]+rad*math.sin(math.radians(rotation+120)), end[1]+rad*math.cos(math.radians(rotation+120)))))


transparent = pygame.Surface((3*width+3*margin,3*height+3*margin))  # the size of your rect
transparent.set_alpha(128)                # alpha level
transparent.fill(DARKGRAY)           # this fills the entire surface

<rect(0, 0, 96, 96)>

In [8]:
def main():
    
    # Loop until the user clicks the close button.
    done = False
    
    s_start=input("Enter start (xnym, where n,m are coordinates) : ")
    s_goal= input("Enter goal (xnym, where n,m are coordinates)  : ")
    #s_start = 'x1y2'
    #s_goal = 'x9y7'

    graph = GridWorld(dimension_x, dimension_y)
    
    goal_coords = stateNameToCoords(s_goal)
    start_coords= stateNameToCoords(s_start)

    graph.setStart(s_start)
    graph.setGoal(s_goal)
          
    k_m = 0
    s_last = s_start
    queue = []

    graph, queue, k_m = initDStarLite(graph, queue, s_start, s_goal, k_m)
    
    initObstacles(graph,obstacle_ratio,fixed_obs_ratio,influence)

    s_current = s_start
    pos_coords = stateNameToCoords(s_current)
    
    print(graph.topography)
    
    robot_centers=[]

    # Initialize pygame
    pygame.init()

    screen = pygame.display.set_mode(WINDOW_SIZE)

    # Set title of screen
    pygame.display.set_caption("D* Lite Path Planning")

    # Used to manage how fast the screen updates
    clock = pygame.time.Clock()
    basicfont = pygame.font.SysFont('Comic Sans MS', 22)

    # -------- Main Program Loop -----------
    while done==False:
        for event in pygame.event.get():   # User did something
            if event.type == pygame.QUIT:  # If user clicked close
                done = True  # Flag that we are done so we exit this loop
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                
                #현재위치가 obstacle한테 잡혔는지 확인
                if s_current in ['x{}y{}'.format(i[0][1],i[0][0]) for i in graph.obstacles]:
                    print('Failed !')
                    done=True
                    break
                
                #아군 먼저 이동
                print('move')
                s_new = Move(graph, s_current)
                if s_new == 'goal':
                    print('Goal Reached!')
                    done = True
                else:
                    # print('setting s_current to ', s_new)
                    '''if s_current == s_new:
                        k_m = Rescan(graph, queue, s_current, k_m)'''
                    '''가만히 있으면 Rescan을 할까?'''
                
                    s_current = s_new
                    pos_coords = stateNameToCoords(s_current)# print('got pos coords: ', pos_coords)
                
                move_obstacle(graph,pos_coords,move_chance=move_chance,smart=smart,detectability=detectability)
                print(graph.cells)

                if step:
                    for obstacle in [obs for obs in graph.obstacles]:
                        
                        loc=obstacle[0]
                        movable=obstacle[2]
                        gap_y = np.abs(pos_coords[1]-loc[0])
                        gap_x = np.abs(pos_coords[0]-loc[1])
                        if movable==0:
                            if gap_y<=1 and gap_x<=1:
                                k_m = Rescan(graph, queue, s_current, k_m)
                                break
                        elif movable==1:
                            if gap_y<=step and gap_x<=step:
                                k_m = Rescan(graph, queue, s_current, k_m)
                                break
                            
                else:
                    k_m = Rescan(graph, queue, s_current, k_m)

            

        # Set the screen background
        screen.fill(BLACK)


        # Draw the grid
        temp_grid=graph.cells.copy()
        temp_grid+=graph.topography#+temp_grid
        temp_grid=np.where(temp_grid<0,-1,temp_grid)
        
                
        for row in range(dimension_y):
            for column in range(dimension_x):
                
                COLOR =colors(temp_grid[row][column])
                if COLOR == RED:
                    if [i[2] for i in graph.obstacles if i[0]==(row,column)][0]==0:
                        COLOR = GRAY
                        
                pygame.draw.rect(screen, COLOR,
                                 [(margin + width) * column + margin,
                                  (margin + height) * row + margin, width, height])
                node_name = 'x' + str(column) + 'y' + str(row)


        # fill in goal cell with GREEN
        pygame.draw.rect(screen, GREEN, [(margin + width) * start_coords[0] + margin,
                                         (margin + height) * start_coords[1] + margin, width, height])


        # fill in goal cell with RED
        pygame.draw.rect(screen, BLUE, [(margin + width) * goal_coords[0] + margin,
                                         (margin + height) * goal_coords[1] + margin, width, height])

        # draw moving robot, based on pos_coords

        # Set the new robot centre
        robot_center = [int(pos_coords[0] * (width + margin) + width / 2) + margin , int(pos_coords[1] * (height + margin) + height / 2) + margin]
        draw_arrow(screen, BLACK, robot_center,robot_center)

        # maintain a list of all cells traversed
        robot_centers.append(robot_center)

        if len(robot_centers)>1:
            for i in range(0,len(robot_centers)-1):
                pygame.draw.line(screen,BLACK,robot_centers[i],robot_centers[i+1],3)

        # grey out visible boxes for robot
        screen.blit(transparent,(robot_center[0]-1.5*width-margin,robot_center[1]-1.5*height-margin))    # (0,0) are the top-left coordinates

        # Limit to 60 frames per second
        clock.tick(20)

        # Go ahead and update the screen with what we've drawn.
        pygame.display.flip()
        

    # Be IDLE friendly. If you forget this line, the program will 'hang'
    # on exit.
    pygame.quit()



if __name__ == "__main__":
    main()

Enter start (xnym, where n,m are coordinates) : x0y0
Enter goal (xnym, where n,m are coordinates)  : x9y9
((7, 5), (0.0, 0.0), 0)
((4, 7), (0.0, 0.0), 0)
((9, 4), (0.0, 0.0), 0)
((8, 1), (0.0, 0.0), 0)
((4, 2), (0.0, 0.0), 0)
((2, 5), (0.0, 0.0), 0)
((0, 4), (0.0, 0.0), 0)
((4, 1), (0.0, 0.0), 0)
((6, 5), (0.3, 0.15), 1)
((1, 9), (0.8, 0.4), 1)
[[0.3, 0.9, 0.3, 0.5, 0.8, 0.1, 0.1, 0.6, 0.7, 0.3], [0.7, 0.3, 0.2, 0.7, 0.6, 0.9, 0.7, 0.2, 0.6, 0.7], [0.7, 0.3, 0.9, 0.6, 0.2, 0.5, 0.6, 0.5, 0.6, 0.1], [0.6, 0.2, 0.8, 0.6, 0.2, 0.3, 0.6, 0.2, 0.8, 0.2], [0.4, 0.2, 0.4, 0.6, 0.4, 0.8, 0.3, 0.5, 0.3, 0.1], [0.6, 0.3, 0.1, 0.1, 0.1, 0.1, 0.7, 0.6, 0.5, 0.3], [0.5, 0.6, 0.2, 0.3, 0.5, 0.8, 0.4, 0.5, 0.6, 0.2], [0.2, 0.7, 0.1, 0.4, 0.5, 0.5, 0.7, 0.4, 0.3, 0.5], [0.6, 0.6, 0.1, 0.9, 0.4, 0.5, 0.7, 0.9, 0.8, 0.5], [0.3, 0.1, 0.5, 0.4, 0.1, 0.8, 0.5, 0.8, 0.3, 0.4]]
move
[[ 0.    0.    0.    0.   -1.    0.    0.4   0.8  -1.    0.8 ]
 [ 0.    0.    0.    0.    0.    0.    0.4   0.8   0.8   0.8 ]
 

2023-06-19 04:28:26.723 python[18273:1511982] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


move
[[ 0.    0.    0.    0.   -1.    0.4   0.8   0.8   0.8   0.4 ]
 [ 0.    0.    0.    0.    0.    0.4   0.8  -1.    0.8   0.4 ]
 [ 0.    0.    0.    0.    0.   -1.    0.8   0.8   0.8   0.4 ]
 [ 0.    0.    0.    0.    0.    0.4   0.4   0.4   0.4   0.4 ]
 [ 0.   -1.   -1.    0.    0.    0.    0.   -1.    0.    0.  ]
 [ 0.    0.    0.15  0.15  0.15  0.15  0.15  0.    0.    0.  ]
 [ 0.    0.    0.15  0.3   0.3   0.3   0.15  0.    0.    0.  ]
 [ 0.    0.    0.15  0.3  -1.   -1.    0.15  0.    0.    0.  ]
 [ 0.   -1.    0.15  0.3   0.3   0.3   0.15  0.    0.    0.  ]
 [ 0.    0.    0.15  0.15 -1.    0.15  0.15  0.    0.    0.  ]]
move
[[ 0.    0.    0.    0.   -1.    0.    0.4   0.4   0.4   0.4 ]
 [ 0.    0.    0.    0.    0.    0.    0.4   0.8   0.8   0.8 ]
 [ 0.    0.    0.    0.    0.   -1.    0.4   0.8  -1.    0.8 ]
 [ 0.    0.    0.    0.    0.    0.    0.4   0.8   0.8   0.8 ]
 [ 0.   -1.   -1.    0.    0.    0.    0.4  -1.    0.4   0.4 ]
 [ 0.    0.15  0.15  0.15  0.15  0.15  0.   

In [7]:
import heapq

move=[(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
#move=[(-1,0),(0,-1),(0,1),(1,0)]

def stateNameToCoords(name):
    return [int(name.split('x')[1].split('y')[0]), int(name.split('x')[1].split('y')[1])]

In [8]:
def iter_test(test_num, dimension_x, dimension_y, s_start, s_goal,
              obstacle_ratio, fixed_obs_ratio, move_chance, 
              step, smart, influence, detectability):

    success=0
    move_num_list=[]
    for idx in range(test_num):
        # Loop until the user clicks the close button.
        done = False

        graph = GridWorld(dimension_x, dimension_y)

        graph.setStart(s_start)
        graph.setGoal(s_goal)

        k_m = 0
        s_last = s_start
        queue = []

        graph, queue, k_m = initDStarLite(graph, queue, s_start, s_goal, k_m)

        initObstacles(graph,obstacle_ratio,fixed_obs_ratio,influence)

        s_current = s_start
        pos_coords = stateNameToCoords(s_current)

        # -------- Main Program Loop -----------
        move_num=0
        while done==False:

            if s_current in ['x{}y{}'.format(i[0][1],i[0][0]) for i in graph.obstacles]:
                print(f'{idx}. Failed !')
                done=True
                break

            s_new = Move(graph, s_current)
            move_num+=1
            if s_new == 'goal':
                done = True
                print(f'{idx}. Success ! --> move {move_num} times')
                move_num_list.append(move_num)
                success+=1
                break
            else:
                # print('setting s_current to ', s_new)
                '''if s_current == s_new:
                    k_m = Rescan(graph, queue, s_current, k_m)'''
                '''가만히 있으면 Rescan을 할까?'''
                s_current = s_new
                pos_coords = stateNameToCoords(s_current)# print('got pos coords: ', pos_coords)

            move_obstacle(graph,pos_coords,move_chance=move_chance,smart=smart,detectability=detectability)

            if step:
                for obstacle in [obs for obs in graph.obstacles]:

                    loc=obstacle[0]
                    movable=obstacle[2]
                    gap_y = np.abs(pos_coords[1]-loc[0])
                    gap_x = np.abs(pos_coords[0]-loc[1])
                    if movable==0:
                        if gap_y<=1 and gap_x<=1:
                            k_m = Rescan(graph, queue, s_current, k_m)
                            break
                    elif movable==1:
                        if gap_y<=step and gap_x<=step:
                            k_m = Rescan(graph, queue, s_current, k_m)
                            break

            else:
                k_m = Rescan(graph, queue, s_current, k_m)

    print('\n-Success Ratio : {}%'.format(100*success/test_num))
    print('-Average moving times : {}'.format(np.mean(move_num_list)))

#### graph size : 10x10, 15x15, 20x20
#### obstacle_ratio : 0.05, 0.1, 0.15, 0.2
#### fixed_obs_ratio : 0.6, 0.7, 0.8, 0.9
#### move_chance : 0.3, 0.5, 0.7, 1.0
#### alpha : 0, 1, 5, 10, 15, 20, 50
#### beta : 0, 1, 3, 5, 7, 10
#### step : 1, 2
#### influence : 1, 2, 3
#### smart : False, True
#### detectability : 1, 5, 10, 30 --smart 일때만 유효

## alpha=0 & beta=0 : 장애물 cost, topography 신경 안씀 --influence 효과 없음

In [9]:
alpha=0
beta=0

In [43]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=1, smart=False, influence=1, detectability=1)

0. Success ! --> move 10 times
1. Success ! --> move 13 times
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
You are done stuck
You are done stuck
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
You are done stuck
You are done stuck
You are done stuck
You are done stuck
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck
empty queue!
You are done stuck

In [44]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=1, smart=True, influence=1, detectability=1)

0. Success ! --> move 10 times
1. Success ! --> move 10 times
2. Success ! --> move 10 times
3. Failed !
4. Failed !
5. Success ! --> move 10 times
6. Success ! --> move 11 times
7. Success ! --> move 11 times
8. Success ! --> move 10 times
9. Success ! --> move 10 times
10. Success ! --> move 10 times
11. Success ! --> move 10 times
12. Success ! --> move 11 times
13. Failed !
14. Success ! --> move 11 times
15. Success ! --> move 10 times
16. Failed !
17. Failed !
18. Failed !
19. Success ! --> move 11 times
20. Success ! --> move 11 times
21. Success ! --> move 10 times
22. Success ! --> move 10 times
23. Success ! --> move 10 times
24. Failed !
25. Failed !
26. Success ! --> move 10 times
27. Success ! --> move 10 times
28. Success ! --> move 10 times
29. Failed !
30. Success ! --> move 10 times
31. Success ! --> move 11 times
32. Success ! --> move 10 times
33. Success ! --> move 11 times
34. Success ! --> move 10 times
35. Success ! --> move 11 times
36. Success ! --> move 10 tim

In [45]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=1, smart=True, influence=1, detectability=5)

0. Failed !
1. Failed !
2. Success ! --> move 10 times
3. Failed !
4. Success ! --> move 10 times
5. Failed !
6. Success ! --> move 11 times
7. Success ! --> move 10 times
8. Success ! --> move 10 times
9. Success ! --> move 11 times
10. Success ! --> move 10 times
11. Failed !
12. Failed !
13. Success ! --> move 10 times
14. Failed !
15. Success ! --> move 10 times
16. Success ! --> move 10 times
17. Success ! --> move 10 times
18. Success ! --> move 10 times
19. Failed !
20. Success ! --> move 10 times
21. Failed !
22. Success ! --> move 11 times
23. Failed !
24. Failed !
25. Success ! --> move 10 times
26. Success ! --> move 11 times
27. Success ! --> move 11 times
28. Failed !
29. Success ! --> move 10 times
30. Success ! --> move 10 times
31. Failed !
32. Success ! --> move 10 times
33. Success ! --> move 10 times
34. Failed !
35. Success ! --> move 10 times
36. Failed !
37. Success ! --> move 11 times
38. Success ! --> move 10 times
39. Success ! --> move 10 times
40. Failed !
41

In [46]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=1, smart=True, influence=1, detectability=10)

0. Failed !
1. Success ! --> move 10 times
2. Success ! --> move 10 times
3. Failed !
4. Success ! --> move 11 times
5. Success ! --> move 11 times
6. Success ! --> move 10 times
7. Success ! --> move 10 times
8. Failed !
9. Success ! --> move 11 times
10. Failed !
11. Success ! --> move 10 times
12. Failed !
13. Failed !
14. Failed !
15. Failed !
16. Success ! --> move 10 times
17. Failed !
18. Failed !
19. Failed !
20. Failed !
21. Success ! --> move 10 times
22. Success ! --> move 10 times
23. Failed !
24. Success ! --> move 10 times
25. Success ! --> move 11 times
26. Success ! --> move 10 times
27. Failed !
28. Success ! --> move 10 times
29. Success ! --> move 10 times
30. Success ! --> move 10 times
31. Success ! --> move 10 times
32. Success ! --> move 10 times
33. Success ! --> move 11 times
34. Success ! --> move 10 times
35. Success ! --> move 11 times
36. Success ! --> move 10 times
37. Success ! --> move 10 times
38. Success ! --> move 10 times
39. Failed !
40. Failed !
41

In [47]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=1, smart=True, influence=1, detectability=30)

0. Success ! --> move 10 times
1. Failed !
2. Failed !
3. Failed !
4. Success ! --> move 10 times
5. Failed !
6. Success ! --> move 11 times
7. Failed !
8. Success ! --> move 10 times
9. Success ! --> move 12 times
10. Success ! --> move 10 times
11. Failed !
12. Success ! --> move 10 times
13. Success ! --> move 10 times
14. Failed !
15. Success ! --> move 10 times
16. Success ! --> move 11 times
17. Success ! --> move 10 times
18. Failed !
19. Failed !
20. Success ! --> move 10 times
21. Failed !
22. Success ! --> move 10 times
23. Success ! --> move 10 times
24. Failed !
25. Failed !
26. Success ! --> move 10 times
27. Success ! --> move 10 times
28. Failed !
29. Failed !
30. Failed !
31. Success ! --> move 10 times
32. Success ! --> move 10 times
33. Failed !
34. Success ! --> move 10 times
35. Failed !
36. Success ! --> move 10 times
37. Success ! --> move 10 times
38. Success ! --> move 10 times
39. Failed !
40. Failed !
41. Success ! --> move 10 times
42. Failed !
43. Success ! 

In [49]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=2, smart=False, influence=1, detectability=1)

0. Success ! --> move 10 times
1. Success ! --> move 10 times
2. Success ! --> move 10 times
3. Success ! --> move 10 times
4. Success ! --> move 11 times
5. Success ! --> move 10 times
6. Success ! --> move 10 times
7. Success ! --> move 10 times
8. Success ! --> move 10 times
9. Success ! --> move 10 times
10. Success ! --> move 10 times
11. Success ! --> move 10 times
12. Success ! --> move 10 times
13. Success ! --> move 10 times
14. Failed !
15. Success ! --> move 10 times
16. Success ! --> move 10 times
17. Success ! --> move 12 times
18. Success ! --> move 10 times
19. Failed !
20. Success ! --> move 10 times
21. Success ! --> move 11 times
22. Failed !
23. Success ! --> move 10 times
24. Success ! --> move 10 times
25. Success ! --> move 10 times
26. Success ! --> move 11 times
27. Failed !
28. Success ! --> move 10 times
29. Success ! --> move 10 times
30. Success ! --> move 10 times
31. Success ! --> move 10 times
32. Success ! --> move 11 times
33. Success ! --> move 11 time

In [50]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=2, smart=True, influence=1, detectability=1)

0. Success ! --> move 10 times
1. Success ! --> move 10 times
2. Success ! --> move 11 times
3. Failed !
4. Success ! --> move 12 times
5. Success ! --> move 10 times
6. Failed !
7. Failed !
8. Failed !
9. Success ! --> move 10 times
10. Failed !
11. Failed !
12. Success ! --> move 10 times
13. Success ! --> move 10 times
14. Success ! --> move 10 times
15. Failed !
16. Success ! --> move 14 times
17. Success ! --> move 10 times
18. Failed !
19. Success ! --> move 10 times
20. Success ! --> move 10 times
21. Failed !
22. Success ! --> move 10 times
23. Success ! --> move 10 times
24. Success ! --> move 10 times
25. Failed !
26. Success ! --> move 10 times
27. Failed !
28. Success ! --> move 11 times
29. Success ! --> move 11 times
30. Failed !
31. Success ! --> move 10 times
32. Failed !
33. Failed !
34. Failed !
35. Success ! --> move 10 times
36. Failed !
37. Failed !
38. Success ! --> move 10 times
39. Success ! --> move 10 times
40. Success ! --> move 10 times
41. Failed !
42. Fail

In [51]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=2, smart=True, influence=1, detectability=5)

0. Success ! --> move 10 times
1. Success ! --> move 11 times
2. Failed !
3. Success ! --> move 10 times
4. Success ! --> move 10 times
5. Success ! --> move 11 times
6. Failed !
7. Failed !
8. Success ! --> move 10 times
9. Success ! --> move 10 times
10. Success ! --> move 10 times
11. Success ! --> move 10 times
12. Success ! --> move 10 times
13. Success ! --> move 10 times
14. Success ! --> move 11 times
15. Failed !
16. Success ! --> move 10 times
17. Success ! --> move 10 times
18. Failed !
19. Success ! --> move 10 times
20. Failed !
21. Success ! --> move 11 times
22. Success ! --> move 10 times
23. Success ! --> move 10 times
24. Success ! --> move 10 times
25. Success ! --> move 10 times
26. Failed !
27. Failed !
28. Failed !
29. Success ! --> move 10 times
30. Failed !
31. Success ! --> move 10 times
32. Success ! --> move 11 times
33. Success ! --> move 11 times
34. Failed !
35. Success ! --> move 11 times
36. Success ! --> move 10 times
37. Failed !
38. Success ! --> move

In [52]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=2, smart=True, influence=1, detectability=10)

0. Success ! --> move 10 times
1. Success ! --> move 11 times
2. Success ! --> move 10 times
3. Success ! --> move 10 times
4. Failed !
5. Success ! --> move 11 times
6. Failed !
7. Failed !
8. Success ! --> move 10 times
9. Failed !
10. Failed !
11. Success ! --> move 10 times
12. Success ! --> move 10 times
13. Success ! --> move 10 times
14. Failed !
15. Success ! --> move 12 times
16. Failed !
17. Success ! --> move 10 times
18. Success ! --> move 10 times
19. Success ! --> move 11 times
20. Success ! --> move 10 times
21. Success ! --> move 10 times
22. Success ! --> move 10 times
23. Success ! --> move 10 times
24. Failed !
25. Success ! --> move 11 times
26. Failed !
27. Success ! --> move 10 times
28. Success ! --> move 10 times
29. Success ! --> move 10 times
30. Failed !
31. Failed !
32. Success ! --> move 10 times
33. Success ! --> move 11 times
34. Failed !
35. Failed !
36. Failed !
37. Failed !
38. Failed !
39. Success ! --> move 10 times
40. Success ! --> move 10 times
41

In [53]:
iter_test(test_num=500, dimension_x=10, dimension_y=10, s_start='x0y0', s_goal='x9y9',
              obstacle_ratio=0.05, fixed_obs_ratio=0.6, move_chance=0.3, 
              step=2, smart=True, influence=1, detectability=30)

0. Failed !
1. Failed !
2. Success ! --> move 10 times
3. Success ! --> move 12 times
4. Failed !
5. Success ! --> move 10 times
6. Failed !
7. Success ! --> move 11 times
8. Success ! --> move 10 times
9. Failed !
10. Success ! --> move 10 times
11. Success ! --> move 10 times
12. Success ! --> move 10 times
13. Success ! --> move 12 times
14. Success ! --> move 10 times
15. Failed !
16. Failed !
17. Failed !
18. Failed !
19. Success ! --> move 10 times
20. Success ! --> move 10 times
21. Failed !
22. Success ! --> move 10 times
23. Success ! --> move 10 times
24. Failed !
25. Failed !
26. Success ! --> move 11 times
27. Success ! --> move 11 times
28. Success ! --> move 11 times
29. Success ! --> move 10 times
30. Success ! --> move 11 times
31. Failed !
32. Failed !
33. Failed !
34. Success ! --> move 10 times
35. Failed !
36. Success ! --> move 10 times
37. Success ! --> move 10 times
38. Failed !
39. Failed !
40. Success ! --> move 11 times
41. Success ! --> move 11 times
42. Fail